# Hugging Face Dataset Loading

In [1]:
!pip install datasets

# RoBERTa Inference

In [2]:
!pip install transformers
!pip install scipy

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

/opt/homebrew/Caskroom/miniforge/base/envs/idl/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
!pip install datasets

In [5]:
def acc(y_true, y_pred):
    count=0
    total=len(y_true)
    for i in range(total):
        if y_true[i]==y_pred[i]:
          count+=1
    return count/total

def f1_value(y_true,y_pred):
    tp=tn=fp=fn=0
    total=len(y_true)
    for i in range(total):
        if y_true[i]==1 and y_pred[i]==1:
          tp+=1
        elif y_true[i]==1 and y_pred[i]==0:
          fn+=1
        elif y_true[i]==0 and y_pred[i]==1:
          fp+=1
        elif y_true[i]==0 and y_pred[i]==0:
          tn+=1
    precision=tp/(tp+fp)
    recall=tp/(tp+fn)
    f1=2*precision*recall/(precision+recall)
    return f1

In [6]:
def preprocessor(sentence):
    words=[]
    for word in sentence.split(' '):
        if word.startswith('@') and len(word) > 1:
            word = '@user'
        
        elif word.startswith('http'):
            word = "http"
        words.append(word)

    new_sentence = " ".join(words)
    return new_sentence


In [7]:
# load model and tokenizer
roberta = "cardiffnlp/twitter-roberta-base-sentiment"

model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)

labels = ['Negative', 'Neutral', 'Positive']

In [8]:
questions=["You're cheering for Quidditch. How do you feel when the team you support wins but it turned out that they used fouls?",
            "You are trying to save Hermione, who was taken by a troll. How do you feel at this time?", 
           "Your team won the Quidditch game, but you found that it was thanks to Harry being attacked by Dementors during the match. In this case, what will you say to whom?",
           "You got on the train to Hogwarts. the only empty seat is next to Luna Lovegood, who is reading with a magazine upside down, wearing a cane in her left ear, and a necklace made of the lid ",
           "You received a turnip earring and a spider ring from Luna Lovegood. How would you feel at this time?"]

answers=[["Still, I don't think it's a good idea to win by foul play",
          "I would get disappointed at them",
          "I would feel not so good because I hope my team win fairly",
          "I would be dissapointed in our team.",
          "I might be disappointed with them.",
          "I feel satisfied."],
         ["I'll gladly help her",
          "I'd get get angry and frustrated",
          "I would feel scared, but not that much. I would concentrate on how to save her from troll.",
          "I will be scared a bit but I know that I have to save her.",
          "I would feel a bit scared but I would focus on rescuing Hermione.",
          "I would be irritated."],
         ["I'll tell the dementors it's wrong",
          "I'll say to Harry that i appreciate him for the sacrifice",
          "I would first say to any of the professors that Harry is under attack. If the game is over, I will wait for the decision whether to have re-match or not",
          "I would say to my team that it's unfair.",
          "I would say to Harry if he's okay and feel sorry to him and his team.",
          "I would tell Harry that he couldn’t help it."],
         ["It's a bit awkward, but I'll have to sit down",
          "She is kind of strange but having interesting traits",
          "Oh, it's interesting. This is a very interesting costume.",
          "I would think her as a strange student and feel a bit uncomfortable.",
          "I would think that she is awkward and I don't want to be close to her.",
          "I would feel irritated."],
         ["I guess I'll answer why she is giving me this.",
          "It's interesting",
          "Why did she give them to me? Should I wear them?",
          "I would feel thank you but will not wear them..",
          "I would feel thankful to her but also think odd at the same time. ",
          "I would feel confused."]]

label_list=[[2,2,2,1,1,4],
            [5,1,3,3,4,3],
            [2,5,3,2,2,2],
            [2,4,4,2,1,2],
            [2,4,1,3,2,1]]


In [9]:
import numpy as np
import time


label=["",'Strong Negative', 'Weak Negative', 'Natural', 'Weak Positive', 'Strong Positive']
n_total=len(questions)*len(answers[0])
avg_time=0
cnt=0
for i in range(len(questions)):
    for j in range(len(answers[i])):
        question=questions[i]
        sentence=answers[i][j]
        start_time=time.time()
        preprocessed=preprocessor(sentence)
        encoded=tokenizer(preprocessed, return_tensors='pt')
        output=model(**encoded)
        scores=output[0][0].detach().numpy()
        scores = softmax(scores)
        diff=time.time()-start_time
        avg_time+=diff
        # print("scores: ", scores)
        # print("argmax: ", np.argmax(scores))
        # print("type: ",type(scores))
        inferenced_class=labels[np.argmax(scores)]
        labeled_class=label[label_list[i][j]]
        print("question        : ", question)
        print("answer          : ", sentence)
        print("labeled         : ", labeled_class)
        print("Inference time  : ", diff)
        print("Inference Result: ", inferenced_class)
        for k in range(len(scores)):
            l = labels[k]
            s = scores[k]
            print(l,s)
        if labeled_class=='Weak Negative' or labeled_class=='Strong Negative':
            labeled_class='Negative'
        elif labeled_class=='Weak Positive' or labeled_class=='Strong Positive':
            labeled_class='Positive'
        elif labeled_class=='Natural':
            labeled_class='Neutral'
        if labeled_class==inferenced_class:
            print("\ninferenced class is same with labeled class !!")
            cnt+=1
        else:
            print("\ninferenced class is different with labeled class !!")
        print("\n\n ======================================= \n\n")

avg_time/=n_total
acc=cnt/n_total
print("average time: ",avg_time)
print("average acc : ",acc)


question        :  You're cheering for Quidditch. How do you feel when the team you support wins but it turned out that they used fouls?
answer          :  Still, I don't think it's a good idea to win by foul play
labeled         :  Weak Negative
Inference time  :  0.20105504989624023
Inference Result:  Negative
Negative 0.8434651
Neutral 0.14988755
Positive 0.0066473624

inferenced class is same with labeled class !!




question        :  You're cheering for Quidditch. How do you feel when the team you support wins but it turned out that they used fouls?
answer          :  I would get disappointed at them
labeled         :  Weak Negative
Inference time  :  0.08087897300720215
Inference Result:  Negative
Negative 0.9642478
Neutral 0.032609753
Positive 0.0031423946

inferenced class is same with labeled class !!




question        :  You're cheering for Quidditch. How do you feel when the team you support wins but it turned out that they used fouls?
answer          :  I would feel not